In [1]:
import os
import pdf2image
from pdf2image import convert_from_path
import cv2
import matplotlib.pyplot as plt
import PyPDF2
import PIL
from PIL import Image, ImageTk
import numpy as np
import pytesseract
from pytesseract import Output
pytesseract.pytesseract.tesseract_cmd="/home/aman/anaconda3/envs/tallyInvoiceParser.env/bin/tesseract"
os.environ['TESSDATA_PREFIX'] = "/home/aman/anaconda3/envs/tallyInvoiceParser.env/share/tessdata"
os.environ['MLIR_CRASH_REPRODUCER_DIRECTORY']='tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269'
import import_ipynb
from textPreProcessor import *
from imagePreProcessor import *
from CreateDatasetForDateModel import *
from datetime import datetime
import math
from keras.models import load_model
import time
import sys
import psutil
from tqdm import tqdm

importing Jupyter notebook from textPreProcessor.ipynb


2023-06-09 14:45:52.150207: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Original Date ----> After Processing 

5/12/22      ---->      5-12-2022
2/2/2023      ---->      2-2-2023
5:12-2023      ---->      5-12-2023
5,May,2023      ---->      5-05-2023
5 May,2023      ---->      5-05-2023
October 24,2022      ---->      10-24-2022
importing Jupyter notebook from imagePreProcessor.ipynb
importing Jupyter notebook from CreateDatasetForDateModel.ipynb


In [2]:
def preProcessImage(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3),1)
    threshold_img = cv2.adaptiveThreshold(blur,255,1,1,11,2)
    return threshold_img

In [3]:
def processForInput(df):
    colsToDrop = ["left", "top", "width", "height", "text", "index", "PageHeight", "PageWidth"]
    d = df.drop(columns=colsToDrop)
    d["conf"]=d["conf"]/100
    d["x"]=d["x"]/1000
    d["y"]=d["y"]/1000
    return d.to_numpy()

In [4]:
def predict(filePath):
    f = open(filePath, 'rb')
    readpdf = PyPDF2.PdfReader(f)
    totalpages = len(readpdf.pages)
    if totalpages==1:
        image = convert_from_path(filePath)
    else:
        return
    image = np.array(image[0])
    image = preProcessImage(image)
    print(type(image))
    data = pytesseract.image_to_data(image, output_type=Output.DATAFRAME)
    df = pd.DataFrame(data)
    preProcessDataFrame(df, image)
    dateDF, dates = extractDateDataFrame(df)
    addNeighbours(df)
    indexes = findDateDF(dates, dateDF)
    resultDF = df.copy()
    dropIndexes(indexes, resultDF)
    resultDF.reset_index(inplace=True)
    texts = []
    inputs = []
    for index in resultDF.index:
        texts.append(resultDF["text"][index])
        inputs.append(processForInput(resultDF))
    predictions = {}
    model = load_model("/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv2/Models/tesseractv2.h5")
    for i in range(len(inputs)):
        text = texts[i]
        input = inputs[i]
        pred = model.predict(input)
        predictions[text]=max(pred)
    if len(predictions)==0:
        return None
    res = max(zip(predictions.values(), predictions.keys()))[1]
    return res
    

In [5]:
os.chdir("/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv2/Dataset/SampleDocuments/")

In [6]:
dir = os.listdir("/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv2/Dataset/SampleDocuments/")

In [7]:
pdfPath = "/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv2/Dataset/SampleDocuments/2223162-TALLY-SOLUTIONS-PVT-LTD.pdf"

In [8]:
print(predict(pdfPath))

<class 'numpy.ndarray'>


<string>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1/1 [==============================] - 0s 14ms/step
9/12/22


In [9]:
def predictImage(imagePath):
    image = cv2.imread(imagePath)
    image = preProcessImage(image)
    data = pytesseract.image_to_data(image, output_type=Output.DATAFRAME)
    df = pd.DataFrame(data)
    preProcessDataFrame(df, image)
    dateDF, dates = extractDateDataFrame(df)
    addNeighbours(df)
    indexes = findDateDF(dates, dateDF)
    resultDF = df.copy()
    dropIndexes(indexes, resultDF)
    resultDF.reset_index(inplace=True)
    texts = []
    inputs = []
    for index in resultDF.index:
        texts.append(resultDF["text"][index])
        inputs.append(processForInput(resultDF))
    predictions = {}
    model = load_model("/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv2/Models/tesseractv2.h5")
    for i in range(len(inputs)):
        text = texts[i]
        input = inputs[i]
        pred = model.predict(input)
        predictions[text]=max(pred)
    if len(predictions)==0:
        return None
    res = max(zip(predictions.values(), predictions.keys()))[1]
    return res

In [10]:
pwd

'/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv2/Dataset/SampleDocuments'

In [11]:
start = time.time()
for i in range(1):
    result = predictImage("/home/aman/Desktop/4.jpeg")
end = time.time()
print(end-start)

<string>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1/1 [==============================] - 0s 36ms/step
1.5089619159698486


In [12]:
print("Invoice Date is : ", result)

Invoice Date is :  12/31/20


In [21]:
def benchmark():
    
    total_steps = 1000
    progress_bar = tqdm(total=total_steps, ncols=80)
    
    n = input("Enter number of iterations : ")
    
    
    start = time.time()
    for i in range(int(n)):
        predictImage("/home/aman/Desktop/4.jpeg")
    end = time.time()
    
    inferenceTime = end-start
    
    cpuUsage = []
    memoryUsage = []
    
    start_time = time.time()
    step=1
    for run in range(10):
        cpuList = []
        memoryList = []
        process = psutil.Process()

        for i in range(10):
            cpuPercent = process.cpu_percent(interval=0.01)
            memory = process.memory_info().rss
            cpuList.append(cpuPercent)
            memoryList.append(memory)

            for j in range(int(n)):
                predictImage("/home/aman/Desktop/4.jpeg")
                
                progress_bar.update(1)
                elapsed_time = time.time() - start_time
                remaining_time = elapsed_time / (step + 1) * (total_steps - step - 1)
                progress_bar.set_postfix({'ETA': progress_bar.format_interval(remaining_time)})
                step+=1
                
        avgCPU = sum(cpuList) / len(cpuList)
        cpuUsage.append(avgCPU)
        avgMemory = sum(memoryList) / len(memoryList)
        memoryUsage.append(avgMemory)
    
    maxCPU = max(cpuUsage)
    minCPU = min(cpuUsage)
    
    maxMemory = max(memoryUsage)
    minMemory = min(memoryUsage)
    
    progress_bar.close()
    
    
    return inferenceTime, maxCPU, minCPU, maxMemory, minMemory
    
    

In [22]:
inferenceTime, maxCPU, minCPU, maxMemory, minMemory = benchmark()





  0%|                                                  | 0/1000 [00:00<?, ?it/s]

Enter number of iterations : 10


<string>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1/1 [==============================] - 0s 37ms/step


<string>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...





100%|████████████████████████████▉| 998/1000 [25:39<00:03,  1.53s/it, ETA=00:03]



100%|████████████████████████████▉| 998/1000 [25:39<00:03,  1.53s/it, ETA=00:01]<string>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1/1 [==============================] - 0s 40ms/step






100%|████████████████████████████▉| 999/1000 [25:41<00:01,  1.53s/it, ETA=00:01]



100%|████████████████████████████▉| 999/1000 [25:41<00:01,  1.53s/it, ETA=00:00]<string>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1/1 [==============================] - 0s 37ms/step






100%|████████████████████████████| 1000/1000 [25:42<00:00,  1.53s/it, ETA=00:00]



100%|█████████████████████████| 1000/1000 [25:42<00:00,  1.54s/it, ETA=-1:59:59]


In [24]:
print("TIME TAKEN : ", inferenceTime)
print("MAX CPU : ", maxCPU, "%")
print("MAX MEMORY : ", maxMemory//(1024*1024), "MB")
print("MIN MEMORY : ", minMemory//(1024*1024), "MB")

TIME TAKEN :  15.427874088287354
MAX CPU :  9.370000000000001 %
MAX MEMORY :  1069.0 MB
MIN MEMORY :  857.0 MB
